In [2]:
import sys
sys.path.append("../src/") 

import model_manipulation as mm


import cobra
import libsbml
import riptide
from cobra import Model, Reaction, Metabolite
import csv
import pandas as pd
import copy

In [9]:
model = cobra.io.read_sbml_model("../model/ios2164_1cell.xml")
rnaseq = pd.read_csv('../data/m_abundance.csv')
mcell_exp = riptide.read_transcription_file("../data/m_abundance.csv", sep = ",", header = True)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
photo_medium = mm.read_medium_csv('../misc/photo_medium.csv', model)

#Define model constraints for photo medium
photo_medium['EX_photonVis(e)'] = 1500
photo_medium['EX_co2(e)'] = 1000
photo_medium['EX_o2(e)'] = 1000
model.medium = photo_medium

model.medium

{'EX_co2(e)': 1000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1500}

In [14]:
mcell_rip = riptide.contextualize(model, transcriptome = mcell_exp, samples= 1000,threshold=1e-9, open_exchanges=True)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Read LP format model from file /tmp/tmp6n82t4v8.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros
Pruning zero flux subnetworks...
Read LP format model from file /tmp/tmpxrbuphb9.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros


/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)


Read LP format model from file /tmp/tmpgcme5uhm.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros
Read LP format model from file /tmp/tmpy3ow8u4e.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros
Read LP format model from file /tmp/tmpyok1rmqr.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros
Read LP format model from file /tmp/tmp6sbt7kxw.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros
Analyzing context-specific flux distributions...
Read LP format model from file /tmp/tmpvr403en0.lp
Reading time = 0.00 seconds
: 454 rows, 914 columns, 3540 nonzeros


/home/rtsantos3/.local/lib/python3.8/site-packages/optlang/interface.py:1474: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self._add_constraints(add_constr)
Could not get flux for reaction FACOAL180c,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction Straw_Biomass,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction ACtex,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction G3P(pi)tsr,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction MDHs,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction 3PG(pi)tsr,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction PPMs,  setting it to NaN. This is usually due to numerical instability.
Could not get flux for reaction PHETAc,  setting it to N

Infeasible: There is no optimal solution for the chosen objective! (infeasible).

In [22]:
for items in model.reactions:
    rxn_id = items.id
    if items.upper_bound > 1e3:
        model.reactions.get_by_id(rxn_id).upper_bound = 1000
    elif items.lower_bound < -1e9:
        model.reactions.get_by_id(rxn_id).lower_bound = -1000
        
for items in model.reactions:
    print(items.id, items.bounds)

Coleoptile_Biomass (0.0, 1000)
Straw_Biomass (0.0, 1000)
ACCOAC_OSAc (-1000, 1000)
ACCOAC_OSAr (-1000, 1000)
ACPC_OSs (-1000, 1000)
ACPC_OSm (-1000, 1000)
FAC_Osc (-1000, 1000)
FAC_Osr (-1000, 1000)
EX_co2(e) (-1000, 0.0)
EX_h2o(e) (-1000000, 1000)
EX_h(e) (0, 1000)
EX_no3(e) (-1000000, 0.0)
EX_o2(e) (-1000, 1000)
EX_pi(e) (-1000000, 1000)
EX_sucr(e) (0, 0.0)
EX_fru-B(e) (0, 0.0)
EX_glc-A(e) (0, 0.0)
EX_so4(e) (-1000000, 1000)
EX_hco3(e) (0, 0.0)
EX_so3(e) (-1000000, 0.0)
EX_h2s(e) (-1000000, 0.0)
EX_nh4(e) (-1000000, 0.0)
EX_asn-L(e) (0, 1000)
EX_gln-L(e) (0, 1000)
EX_etoh(e) (0, 1000)
EX_ac(e) (0, 1000)
EX_lac-L(e) (0, 1000)
EX_tsul(e) (0, 0.0)
EX_fe2(e) (-1000000, 1000)
EX_fe3(e) (-1000000, 0.0)
EX_mg2(e) (-1000000, 1000)
EX_ala-L(e) (0, 1000)
EX_arg-L(e) (0, 1000)
EX_asp-L(e) (0, 1000)
EX_cys-L(e) (0, 1000)
EX_glu-L(e) (0, 1000)
EX_gly(e) (0, 1000)
EX_his-L(e) (0, 1000)
EX_ile-L(e) (0, 1000)
EX_leu-L(e) (0, 1000)
EX_lys-L(e) (0, 1000)
EX_met-L(e) (0, 1000)
EX_phe-L(e) (0, 1000)
EX_

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
print(model.objective)

Maximize
1.0*Straw_Biomass - 1.0*Straw_Biomass_reverse_5daed


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
sample_fluxes = cobra.flux_analysis.pfba(mcell_rip.model)
sample_fluxes

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Infeasible: None (infeasible).

In [12]:
mcell_model = mcell_rip.model

for exchs in mcell_model.reactions:
    try:
        print(exchs.id, sample_fluxes[exchs.id])
        
    except KeyError:
        pass


Straw_Biomass 10.162620324549401
ACCOAC_OSAr 1.0260384732071568
FAC_Osr -0.028963467924965795
EX_co2(e) -43.35986143274459
EX_h2o(e) 1000.0
EX_o2(e) -737.209677085612
EX_pi(e) -2.0971888180352405
EX_so4(e) 425.98035600906474
EX_so3(e) -425.98035600906474
EX_h2s(e) -0.11077256153758874
EX_nh4(e) -6.514036375629277
EX_photonVis(e) -1500.0
PRISM_blue_LED 1000.0
PRISM_red_LED 500.0
RPSI_430(u) 0.0
RPSI_610(u) 1.439788
RPSI_630(u) 1.9276900965
RPSI_650(u) 241.6787645
RPSII_410(u) 0.9290386222
RPSII_430(u) 370.517176
RPSII_450(u) 520.4014080000001
RPSII_470(u) 94.361569
RPSII_490(u) 4.578975509999999
RPSII_670(u) 228.28538400000002
RPSII_690(u) 5.414752
CO2tex 43.35986143274459
H2Otex -1000.0
O2tex 737.209677085612
Pitex 2.0971888180352405
SO4tex -425.98035600906474
SO3tex 425.98035600906474
H2Stex 0.11077256153758874
NH4tex 6.514036375629277
PPASc 0.0
PPASs 13.786610732283718
PPAIcxc 122.49382594054521
PPAIcvc 2.2743944282654853
APLm 0.4664642728968175
CATv 212.99017800453237
RBPCs 540.2086

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
mcell_model.reactions.get_by_id('RBPCs').summary()

In [13]:
mcell_model.reactions.RBPCs

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reaction identifier,RBPCs
Name,Ribulose-bisphosphate carboxylase
Memory address,0x07f82adfe0d60
Stoichiometry,"co2_s + h2o_s + rb15bp_s --> 2.0 3pg_s + h_s CO2 + H2O + D-Ribulose 1,5-bisphosphate --> 2.0 3-Phospho-D-glycerate + H+"
GPR,LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280 and...
Lower bound,432.16695000671376
Upper bound,540.2086875083758


In [ ]:
mcell_fva = mcell_rip.flux_variability

In [17]:
mcell_model.reactions.RBPCs

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reaction identifier,RBPCs
Name,Ribulose-bisphosphate carboxylase
Memory address,0x07f82adfe0d60
Stoichiometry,"co2_s + h2o_s + rb15bp_s --> 2.0 3pg_s + h_s CO2 + H2O + D-Ribulose 1,5-bisphosphate --> 2.0 3-Phospho-D-glycerate + H+"
GPR,LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280 and...
Lower bound,432.16695000671376
Upper bound,540.2086875083758


In [20]:
mcell_model.reactions.PPCc.bounds

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(11.43005392808584, 14.287567410107727)

Benchmarking Results for M cells:


Unconstrained CO2 Flux thru M: r=0.077, n.s.
Orig. Medium (Photoautotrophic, w/ Gas exchange): r=0.192, p<0.001
If I add Sucrose to the medium: r=0.065; p=0.045 (Low significance)
Only CO2, no O2: 0.154, p=0.001
With NH4 + Gas exchange: r=0.181, p<0.001
With NH4 + Gas exchange + AA: r=0.15, p<0.001
Low light conditions (200 PPFD), gas exchange + unconstrained inorganics = 0.171
Medium light (500 PPFD): 0.15
With Glycine import: 0.162

W/ FE3 import, no NH4: r=0.213, p<0.001

Just use yung with FE3 import and no NH4. //